In [1]:
# ============================================================
# Notebook setup: run this before everything
# ============================================================
# -- Copied from lecture
%load_ext autoreload
%config IPCompleter.greedy=True
%autoreload 1
%aimport util
import logging

import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.neighbors import KernelDensity
from sklearn.preprocessing import StandardScaler

from util import util

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Control figure size
interactive_figures = False
if interactive_figures:
    # Normal behavior
    %matplotlib widget
    figsize=(9, 3)
else:
    # PDF export behavior
    figsize=(14, 4)

# Multivariate Kernel Density Estimation
The first approach presented in the lecture is **Kernel Density Estimation**

In order to employ **KDE**, we need to determine the optimal **Kernel Function** and **Bandwidth**.
Since we have multiple columns, we cannot use the Rule Of Thumb for the latter. Therefore, we need to optimize the following term according to the lecture:
$$
\mathop{\arg\max}_{h} \mathbb{E}_{x \sim f(x), \bar{x} \sim f(x)}\left[ L(h, x, \bar{x})\right]
$$
where
- $$
L(h, x, \bar{x}) = \prod_{i=1}^m \hat{f}(x_i, \bar{x}_i, h)
$$
- $\hat{f}$ is the density estimator (which outputs a probability)
- $\bar{x}$ the training set

according to the lecture.

## Preprocessing
KDE is a non-parametric method, meaning that it does not make any assumptions about the shape and the distribution of the data. Therefore, we need to preprocess the data before applying KDE. First, we have to make sure that our data is free from missing values. As seen before, a linear interpolation approach yields the best results. Therefore, we will interpolate missing values using this method. Then, we have to apply a sliding window approach using the `aggregation_length` parameter explained before and aggregate the data into windows. This makes sure that we capture temporal correlations between data points and additionally removes noise in the data. The final step of our preprocessing pipeline is to standardize the data.

In [2]:
# Preprocess config
preprocess_clean = [util.impute_missing_values, util.impute_anomalies, util.apply_sliding_window_and_aggregate]
preprocess = [util.impute_missing_values, util.apply_sliding_window_and_aggregate]

# Load datasets
X_train_clean, y_train_clean = util.load_dataset_xy('7_gecco2019_train_water_quality.csv', preprocess=preprocess_clean)
X_train, y_train = util.load_dataset_xy('7_gecco2019_train_water_quality.csv', preprocess=preprocess)
X_val, y_val = util.load_dataset_xy('8_gecco2019_valid_water_quality.csv', preprocess=preprocess)
X_test, y_test = util.load_dataset_xy('6_gecco2019_test_water_quality.csv', preprocess=preprocess)

# Identify the features to be used for KDE
features = util.get_feature_columns(X_train_clean)

# Standardize the data (KDE assumes normally distributed data)
scaler = StandardScaler()
X_train_clean[features] = scaler.fit_transform(X_train_clean[features])
X_train[features] = scaler.transform(X_train[features])
X_val[features] = scaler.transform(X_val[features])
X_test[features] = scaler.transform(X_test[features])

print(X_train_clean.head())

                     window_0  window_1  window_2  window_3  window_4  \
Time                                                                    
2017-07-01 00:09:00 -1.232383  1.187793 -0.392840 -0.809330 -1.697608   
2017-07-01 00:10:00 -1.243568  1.158432 -0.359859 -0.372582 -1.686283   
2017-07-01 00:11:00 -1.232383  1.099867 -0.352313 -0.622491 -1.709695   
2017-07-01 00:12:00 -1.232383  1.099867 -0.342251 -0.723540 -1.694760   
2017-07-01 00:13:00 -1.232383  1.129229 -0.337220 -0.138949 -1.710721   

                     window_5  window_6  window_7  window_8  window_9  ...  \
Time                                                                   ...   
2017-07-01 00:09:00 -2.181413 -1.243574  1.158442 -0.359853 -0.372584  ...   
2017-07-01 00:10:00 -2.209374 -1.232388  1.099877 -0.352306 -0.622493  ...   
2017-07-01 00:11:00 -2.195118 -1.232388  1.099877 -0.342244 -0.723542  ...   
2017-07-01 00:12:00 -2.215171 -1.232388  1.129239 -0.337213 -0.138951  ...   
2017-07-01 00:13:00 

## Bandwidth Choice in Multivariate KDE
The bandwidth parameter $h$ is a hyperparameter that controls the smoothness of the estimated density. It has to be chosen such that it maximizes the likelihood of the data:

$$
argmax_h \mathbb{E}_{x \sim f(x), \bar{x} \sim f(x)}\left[ L(h, x, \bar{x})\right],
$$

where $f(x)$ is the true distribution function and $L(h, x, \bar{x)$ is the log-likelihood function.

 In the lecture, we discussed the following methods to determine the optimal bandwidth:
- The rule of thumb method: This method uses a rule of thumb to estimate the optimal bandwidth based on the data. It involves calculating the standard deviation of the data and using it as the bandwidth. However, this method is not always reliable, as it can be sensitive to the scale of the data.
- Cross-validation: This method involves splitting the data into training and validation sets and using the validation set to estimate the optimal bandwidth. The model is then trained on the training set and evaluated on the validation set. This method is more robust than the rule of thumb method, as it can handle non-normal data and avoid overfitting.
- Grid search: This method involves searching over a grid of possible bandwidths and selecting the one that gives the best performance. This method is computationally expensive, but it can be useful when the data is not normally distributed.

In our case, we will use a grid search to determine the optimal bandwidth. We will use the F1-score to evaluate the performance of the KDE model using a given bandwidth. The grid search will be performed using the `sklearn.model_selection.GridSearchCV` class. As we're working with time-series data, we will use a `sklearn.model_selection.TimeSeriesSplit` cross-validation strategy to split the data into training and validation sets.

The downside of the grid search strategy is that it is computationally very expensive, especially for large datasets, as it runs a cross-validation for each possible bandwidth value.

In [3]:
# Define a reasonable range of bandwidths
bandwidths = np.linspace(0.7, 0.8, 5)

# Create a TimeSeriesSplit instance to ensure training data precedes test data
tscv = TimeSeriesSplit(n_splits=5)

# Set up GridSearchCV with verbose output to monitor progress; n_jobs=-1 uses all cores
grid = GridSearchCV(KernelDensity(kernel='gaussian'), {'bandwidth': bandwidths}, cv=tscv, verbose=3, n_jobs=-1)

# Run the grid search to fit the KDE model and determine the best bandwidth
grid.fit(X_train_clean)

best_bw = grid.best_params_['bandwidth']
print("Best bandwidth:", grid.best_params_['bandwidth'])
print("Best log-likelihood score:", grid.best_score_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV 3/5] END ..............bandwidth=0.7;, score=-2857078.549 total time= 5.9min
[CV 1/5] END ............bandwidth=0.775;, score=-1383014.852 total time= 2.4min
[CV 1/5] END ..............bandwidth=0.8;, score=-1418061.428 total time= 2.5min
[CV 2/5] END ............bandwidth=0.725;, score=-1329128.011 total time= 4.5min
[CV 4/5] END .............bandwidth=0.75;, score=-1291182.083 total time= 8.0min
Best bandwidth: 0.7
Best log-likelihood score: -1707475.8147475843


## Train the KDE
Next, we train the KDE with the optimal bandwidth. Additionally, we compute the log likelihood scores for each data point. This score is a measure of how likely a data point is to be generated by the underlying distribution. Higher scores indicate a higher likelihood of being generated by a Gaussian distribution.

Due to the high dimensionality of the data, training and evaluating the KDE model suffer from the "curse of dimensionality". This means that the model becomes increasingly difficult to train and evaluate as the number of dimensions increases.

In [4]:
# Train KDE with best bandwidth.
final_kde = KernelDensity(kernel='gaussian', bandwidth=best_bw)
final_kde.fit(X_train_clean)

KernelDensity(bandwidth=np.float64(0.7))

## Threshold Optimization
Now, we need to define a threshold to separate normal data from anomalous data. We will use a simple threshold optimization approach using the validation set. First, we define the percentiles to test. Then, we compute precision, recall, and F1-score for each percentile. These are the preferred metrics when working with big class imbalances. Finally, we select the percentile with the highest F1-score.

In [5]:
# Compute scores for validation data
log_likelihood_val = final_kde.score_samples(X_val)

# Define percentiles to test.
percentiles = np.arange(0.1, 2.1, 0.1)

# For storing the results.
results = []

for p in percentiles:
    # Get predictions and threshold
    y_pred, threshold = util.get_predictions_from_log_likelihood(log_likelihood_val, p)

    # Compute performance
    f1, precision, recall = util.compute_model_performance(y_pred, y_val)

    # Store results.
    results.append((p, threshold, precision, recall, f1))

# Convert to DataFrame for better visualization.
df_results = pd.DataFrame(results, columns=['Percentile', 'Threshold', 'Precision', 'Recall', 'F1-score'])

# Display results.
best_percentile = df_results.loc[df_results['F1-score'].idxmax()]
print(df_results)
print(f"Best KDE model with percentile {best_percentile['Percentile']} and threshold {best_percentile['Threshold']} achieves an F1-score of {best_percentile['F1-score']}")

[CV 5/5] END ..............bandwidth=0.7;, score=-1904169.981 total time= 8.7min
[CV 2/5] END ..............bandwidth=0.8;, score=-1436417.615 total time= 4.4min
[CV 1/5] END ............bandwidth=0.725;, score=-1312093.990 total time= 2.3min
[CV 2/5] END .............bandwidth=0.75;, score=-1362568.952 total time= 4.6min
[CV 3/5] END ............bandwidth=0.775;, score=-2683375.531 total time= 6.2min
[CV 1/5] END ..............bandwidth=0.7;, score=-1275947.966 total time= 2.2min
[CV 1/5] END .............bandwidth=0.75;, score=-1347563.961 total time= 2.4min
[CV 5/5] END .............bandwidth=0.75;, score=-1917449.333 total time= 8.6min
[CV 5/5] END ............bandwidth=0.725;, score=-1928330.635 total time= 8.8min
[CV 3/5] END ..............bandwidth=0.8;, score=-2638394.397 total time= 5.8min
[CV 4/5] END ..............bandwidth=0.7;, score=-1205810.678 total time= 7.6min
[CV 4/5] END ............bandwidth=0.775;, score=-1333362.719 total time= 7.5min
[CV 4/5] END ............ban

## Performance on Training Data
We will determine the model performance by investigating the confusion matrix and the classification report. Both offer valuable insights on the success of the model's training.

In [6]:
# Compute likelihood scores for the training data.
log_likelihood_train = final_kde.score_samples(X_train)
y_pred_train, threshold_train = util.get_predictions_from_log_likelihood(log_likelihood_train, best_percentile['Percentile'])

# Print threshold
print(f'Ideal threshold: {threshold_train:.2f}')

# Print performance metrics
print("Confusion Matrix:\n", confusion_matrix(y_train, y_pred_train))
print("Classification Report:\n", classification_report(y_train, y_pred_train))

Ideal threshold: -1124.40
Confusion Matrix:
 [[132074     68]
 [   264     65]]
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    132142
         1.0       0.49      0.20      0.28       329

    accuracy                           1.00    132471
   macro avg       0.74      0.60      0.64    132471
weighted avg       1.00      1.00      1.00    132471



The model achieves a very high overall accuracy of nearly 100%, driven primarily by its excellent performance on the dominant class (class 0), where it correctly identifies almost all cases (approximately 99.95% recall and nearly 100% precision). However, this high accuracy masks a significant weakness in detecting the minority class (class 1). For class 1, the recall is very low at about 20% which indicates that only one in five actual positives is captured. The precision is moderately better at around 49%, indicating that roughly half of the flagged cases are true positives. This imbalance suggests that while the KDE model is extremely effective at recognizing the majority class, it struggles to reliably identify the minority class, likely due to the imbalanced nature of the training data.

## Performance on Test Data
Finally, we evaluate the model's performance on the test data. This is an important step that allows us to assess how well the model generalizes to unseen data. We simply obtain the log likelihoods for test data and compute the confusion matrix and classification report.

In [7]:
# Compute likelihood scores for the test data.
log_likelihood_test = final_kde.score_samples(X_test)
y_pred_test, threshold_test = util.get_predictions_from_log_likelihood(log_likelihood_test, best_percentile['Percentile'])

# Print performance metrics
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred_test))
print("Classification Report:\n", classification_report(y_test, y_pred_test))

Confusion Matrix:
 [[31337     5]
 [  272    27]]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.99      1.00      1.00     31342
         1.0       0.84      0.09      0.16       299

    accuracy                           0.99     31641
   macro avg       0.92      0.55      0.58     31641
weighted avg       0.99      0.99      0.99     31641



On the test data, the model achieves an overall accuracy of 99% and performs exceptionally well for the majority class (class 0) with a precision of 0.99 and a perfect recall of 1.00. However, its performance on the minority class (class 1) is concerning. The model correctly identifies only 27 out of 299 class 1 instances, resulting in a very low recall of 9%, even though the precision for class 1 is relatively high at 84%. This discrepancy indicates that while the model is likely correct when it predicts class 1, it misses a significant number of true positives. Overall, the F1-score has decreased and is now even lower at 0.16 while it was 0.28 on the training data. These differences indicate a potential problem of overfitting or that the model's ability to generalize to the minority class is limited.